# **<font color='crimson'>SQL. Subqueries</font>**

---

In [ ]:
###

In [ ]:
--1
SELECT
    ROUND(AVG(t.count_orders), 2) AS orders_avg
FROM
    (
    SELECT
        t.user_id,
        COUNT(DISTINCT t.order_id) AS count_orders
    FROM
        user_actions AS t
    WHERE
        t.action = 'create_order'
    GROUP BY
        t.user_id) AS t

In [ ]:
###

In [ ]:
--2
WITH subquery AS (
    SELECT
        t.user_id,
        COUNT(DISTINCT t.order_id) AS count_orders
    FROM
        user_actions AS t
    WHERE
        t.action = 'create_order'
    GROUP BY
        t.user_id)

SELECT
    ROUND(AVG(t.count_orders), 2) AS orders_avg
FROM
    subquery AS t;

In [ ]:
###

In [ ]:
--3
WITH subquery AS (
    SELECT
        MIN(t.price) AS min_price
    FROM
        products AS t
)

SELECT
    t.product_id,
    t.name,
    t.price
FROM
    products AS t
WHERE
    price <> (SELECT min_price FROM subquery)
ORDER BY
    t.product_id DESC;

In [ ]:
###

In [ ]:
--4
WITH subquery AS (
    SELECT
        AVG(t.price) AS avg_price
    FROM
        products AS t)

SELECT
    t.product_id
    ,t.name
    ,t.price
FROM
    products AS t
WHERE
    t.price  > (SELECT avg_price FROM subquery) + 20
ORDER BY
    t.product_id DESC;

In [ ]:
###

In [ ]:
--5
WITH subquery AS (
    SELECT
        MAX(t.time) AS max_time
    FROM
        user_actions AS t
)

SELECT
    COUNT(DISTINCT t.user_id) AS users_count
FROM
    user_actions AS t
WHERE
    t.action = 'create_order'
    AND (t.time BETWEEN
        (SELECT t.max_time FROM subquery AS t) - INTERVAL '7 days'
        AND (SELECT t.max_time FROM subquery AS t));

In [ ]:
###

In [ ]:
--6
WITH subquery AS (
    SELECT
        MAX(t.time) AS last_date
    FROM
        courier_actions AS t
)

SELECT
    MIN(AGE(
        (SELECT t.last_date FROM subquery AS t)::DATE,
        t.birth_date))::VARCHAR AS min_age
FROM
    couriers AS t
WHERE
    sex = 'male';

In [ ]:
###

In [ ]:
--7
WITH subquery AS (
    SELECT
        t.order_id
    FROM
        user_actions AS t
    WHERE
        t.action = 'cancel_order'
)

SELECT
    t.order_id
FROM
    user_actions AS t
WHERE t.order_id NOT IN (SELECT t.order_id FROM subquery As t)
ORDER BY
    t.order_id ASC
LIMIT 1000;

In [ ]:
###

In [ ]:
--8
WITH subquery_1 AS (
    SELECT
        t.user_id
        ,COUNT(t.order_id) AS orders_count
    FROM
        user_actions AS t
    WHERE
        t.action = 'create_order'
    GROUP BY t.user_id
    ORDER BY t.user_id ASC
),
subquery_2 AS (
    SELECT
        ROUND(AVG(t.orders_count), 2) AS orders_avg
    FROM
        subquery_1 AS t
)

SELECT
    t.user_id
    ,t.orders_count
    ,(SELECT t.orders_avg FROM subquery_2 AS t) AS orders_avg
    ,t.orders_count - (SELECT t.orders_avg FROM subquery_2 AS t) AS orders_diff
FROM
    subquery_1 AS t
ORDER BY t.user_id ASC
LIMIT 1000;

In [ ]:
###

In [ ]:
--9
WITH subquery AS (
    SELECT
        ROUND(AVG(t.price), 2) AS average_price
    FROM
        products AS t
)

SELECT
    t.product_id
    ,t.name
    ,t.price
    ,CASE
    WHEN t.price > (
        SELECT t.average_price FROM subquery AS t) + 50 THEN t.price * 0.85
    WHEN t.price < (
        SELECT t.average_price FROM subquery AS t) - 50 THEN t.price * 0.90
    ELSE t.price
    END AS new_price
FROM
    products AS t
ORDER BY
    t.price DESC
    ,t.product_id ASC;

In [ ]:
###

In [ ]:
--10
WITH subquery AS (
    SELECT
        t.order_id
    FROM
        user_actions AS t
    WHERE
        t.action  = 'create_order'
)

SELECT
    COUNT(DISTINCT t.order_id) AS orders_count
FROM
    courier_actions AS t
WHERE
    t.order_id NOT IN (
        SELECT t.order_id
        FROM subquery AS t);

In [ ]:
###

In [ ]:
--11
WITH subquery AS (
    SELECT
        t.order_id
    FROM
        courier_actions AS t
    WHERE
        t.action = 'deliver_order'
)

SELECT
    COUNT(DISTINCT t.order_id) AS orders_count
FROM
    courier_actions AS t
WHERE
    t.order_id NOT IN (
    SELECT t.order_id FROM subquery AS t
);

In [ ]:
###

In [ ]:
--12
WITH subquery AS (
    SELECT
        t.order_id AS order_id_cancel
    FROM
        user_actions AS t
    WHERE
        t.action = 'cancel_order'
)

SELECT
    COUNT(DISTINCT t.order_id) FILTER (
        WHERE t.order_id IN (
            SELECT t.order_id_cancel FROM subquery AS t))
        AS orders_canceled
    ,COUNT(DISTINCT t.order_id) FILTER (
        WHERE t.action = 'deliver_order'
        AND t.order_id IN (
            SELECT t.order_id_cancel FROM subquery AS t))
        AS orders_canceled_and_delivered
FROM
    courier_actions AS t;

In [ ]:
###

In [ ]:
--13
WITH id_deliver AS (
    SELECT
        t.order_id AS order_id_deliver
    FROM
        courier_actions AS t
    WHERE t.action = 'deliver_order'
)

SELECT
    COUNT(DISTINCT t.order_id) FILTER (
        WHERE t.order_id NOT IN (
            SELECT t.order_id_deliver FROM id_deliver AS t))
        AS orders_undelivered

    ,COUNT(DISTINCT t.order_id) FILTER (
        WHERE t.order_id IN(
            SELECT t.order_id FROM user_actions AS t
            WHERE t.action = 'cancel_order'))
        AS orders_canceled

    ,COUNT(DISTINCT t.order_id) FILTER (
        WHERE t.order_id NOT IN (
            SELECT t.order_id FROM user_actions AS t
            WHERE t.action = 'cancel_order')
            AND t.order_id NOT IN (
                SELECT t.order_id_deliver FROM id_deliver AS t))
        AS orders_in_process
FROM
    courier_actions AS t;

In [ ]:
###

In [ ]:
--14
WITH subquery AS (
    SELECT
        MIN(t.birth_date) AS min_female_date
    FROM
        users AS t
    WHERE
        t.sex = 'female'
)

SELECT
    t.user_id
    ,t.birth_date
FROM
    users AS t
WHERE
    t.sex = 'male'
    AND t.birth_date < (
        SELECT t.min_female_date
        FROM subquery AS t)
ORDER BY t.user_id ASC;

In [ ]:
###

In [ ]:
--15
WITH subquery As (
    SELECT
        t.order_id AS order_id_deliver
    FROM
        courier_actions AS t
    WHERE
        t.action = 'deliver_order'
    ORDER BY
        t.time DESC
    LIMIT 100
)

SELECT
    t.order_id,
    t.product_ids
FROM
    orders AS t
WHERE
    t.order_id IN (
        SELECT t.order_id_deliver FROM subquery AS t)
ORDER BY
    t.order_id ASC;

In [ ]:
###

In [ ]:
--16
WITH subquery AS (
    SELECT
        t.courier_id AS leader_courier_id
    FROM
        courier_actions AS t
    WHERE
        DATE_PART('year', t.time) = 2022
        AND DATE_PART('month', t.time) = 9
        AND t.action = 'deliver_order'
    GROUP BY
        t.courier_id
    HAVING
        COUNT(DISTINCT t.order_id) >= 30
)

SELECT
    t.courier_id
    ,t.birth_date
    ,t.sex
FROM
    couriers AS t
WHERE
    t.courier_id IN (
        SELECT t.leader_courier_id
        FROM subquery AS t)
ORDER BY
    t.courier_id ASC;

In [ ]:
###

In [ ]:
--17
--находим id клиентов мужского пола
WITH user_id_male AS (
    SELECT
        t.user_id
    FROM
        users AS t
    WHERE
        t.sex = 'male'
),
--находим id заказов, отмененных
--клиентами мужского пола
order_id_cancel_male AS (
    SELECT
        t.order_id AS order_id_cancel
    FROM
        user_actions AS t
    WHERE
        t.action = 'cancel_order'
        AND t.user_id IN (
            SELECT t.user_id
            FROM user_id_male AS t)
)
--рассчитываем средний размер заказов,
--отмененных пользователями мужского пола
SELECT
    ROUND(AVG(ARRAY_LENGTH(t.product_ids, 1)), 3) AS avg_order_size
FROM
    orders AS t
WHERE
    t.order_id IN (
        SELECT t.order_id_cancel
        FROM order_id_cancel_male AS t);

In [ ]:
###

In [ ]:
--18
--определим дату и время самого последнего действия пользователя
--в отчетных документах
WITH max_date AS (
    SELECT
        MAX(t.time) AS max_time
    FROM
        user_actions AS t
),
--вычислим средний возраст пользователей
--по пользователям, у которых возраст указан
avg_age AS (
    SELECT
    ROUND(AVG(DATE_PART('year', AGE(
        (SELECT MAX(t.time) FROM user_actions AS t),
        t.birth_date)))) AS average_age
    FROM users AS t
)
--вычислим возраст каждого пользователя
--относительно последней даты активности,
--регистрируемой в отчетных документах;
--пропуски заполним средним значением возраста
SELECT
    t.user_id
    ,COALESCE(
        (DATE_PART('year', AGE(
        (SELECT t.max_time FROM max_date AS t),
        t.birth_date))),
        (SELECT t.average_age FROM avg_age AS t))::int AS age
FROM
    users AS t
ORDER BY
    t.user_id ASC;

In [ ]:
###

In [ ]:
--19
--определим id отмененных заказов
WITH order_canceled AS (
    SELECT
        t.order_id AS order_id_cancel
    FROM
        user_actions AS t
    WHERE
        t.action = 'cancel_order'
),
--определим id заказов, в которые входит более пяти товаров
--и которые не были отменены
more_then_five_products_ids AS(
    SELECT
        t.order_id
    FROM
        orders AS t
    WHERE
        ARRAY_LENGTH(t.product_ids, 1) > 5
        AND t.order_id NOT IN (
            SELECT t.order_id_cancel
            FROM order_canceled AS t)
),
--отфильтруем время принятия заказа
order_accepted AS (
    SELECT
        t.order_id
        ,t.time AS time_accepted
    FROM
        courier_actions AS t
    WHERE
        t.action = 'accept_order'
        AND t.order_id IN (
            SELECT t.order_id FROM more_then_five_products_ids AS t)
),
--отфильтруем время доставки заказа
order_delivered AS (
    SELECT
        t.order_id
        ,t.time AS time_delivered
    FROM
        courier_actions AS t
    WHERE
        t.action = 'deliver_order'
        AND t.order_id IN (
            SELECT t.order_id FROM more_then_five_products_ids AS t)
),
--объеденим id заказов, соответтсвующих условию задачи
--со временем принятия и доставки заказа
joined_table AS (
    SELECT
        t.order_id
        ,order_accepted.time_accepted
        ,order_delivered.time_delivered
    FROM
        courier_actions AS t
        JOIN order_accepted
            ON t.order_id = order_accepted.order_id
        JOIN order_delivered
            ON order_accepted.order_id = order_delivered.order_id
    WHERE
        t.order_id IN
           (SELECT t.order_id FROM more_then_five_products_ids AS t)
    GROUP BY
        t.order_id
        ,order_accepted.time_accepted
        ,order_delivered.time_delivered
    )
    --вычислим время доставки каждого заказа
    SELECT
        t.order_id
        ,t.time_accepted
        ,t.time_delivered
        ,ROUND(EXTRACT(EPOCH FROM (t.time_delivered - t.time_accepted)) / 60
               )::INTEGER As delivery_time
    FROM
        joined_table AS t
ORDER BY
    t.order_id ASC;

In [ ]:
--получаем id заказов, в которых более 5 товаров
WITH big_orders AS (
    SELECT
        t.order_id
    FROM
        orders AS t
    WHERE
        ARRAY_LENGTH(t.product_ids, 1) > 5
),
--получаем id отмененных клиентами заказов
orders_cancel AS (
    SELECT
        t.order_id AS order_id_cancel
    FROM
        user_actions AS t
    WHERE
        t.action = 'cancel_order'
)
--для каждого заказа с количеством товаров в заказе более 5,
--который не был отменен,
--рассчитаем время доставки в минутах
SELECT
    t.order_id
    ,MIN(t.time) AS time_accepted
    ,MAX(t.time) AS time_delivered
    ,ROUND(EXTRACT(EPOCH FROM MAX(t.time) - MIN(t.time)) / 60
        )::INTEGER AS delivery_time
FROM
    courier_actions AS t
WHERE
    t.order_id IN (
        SELECT t.order_id FROM big_orders AS t)
    AND t.order_id NOT IN (
        SELECT t.order_id_cancel FROM orders_cancel AS t)
GROUP BY
    t.order_id
ORDER BY
    t.order_id ASC;

In [ ]:
###

In [ ]:
--20
--находим id отмененных заказов
WITH orders_cancel AS (
    SELECT
        t.order_id AS orders_id_cancel
    FROM
        user_actions AS t
    WHERE
        t.action = 'cancel_order'
),
--находим время первого заказа каждого пользователя
--для неотмененных заказов
first_order_date AS (
    SELECT
        MIN(t.time::DATE) AS date
    FROM
        user_actions AS t
    WHERE
        t.order_id NOT IN (
            SELECT t.orders_id_cancel FROM orders_cancel AS t)
    GROUP BY
        t.user_id
)
--вычисляем количество первых заказов пользоваелей по дням
SELECT
    t.date
    ,COUNT(t.date) AS first_orders
FROM
    first_order_date AS t
GROUP BY
    t.date
ORDER BY
    t.date ASC;

In [ ]:
###

In [ ]:
--21
SELECT
    t.order_id
    ,t.creation_time
    ,t.product_ids
    ,UNNEST(product_ids) AS product_id
FROM
    orders AS t
LIMIT 100;

In [ ]:
###

In [ ]:
--22
--получим id отмененных заказов
WITH order_id_cancel AS (
SELECT
    t.order_id
FROM
    user_actions AS t
WHERE
    t.action = 'cancel_order'
),
--трансформируем списки с id заказанных товаров,
--содержащиеся в признаке product_ids,
--исключив отмененные заказы
unnest_table AS (
    SELECT
        t.order_id
        ,t.creation_time
        ,t.product_ids
        ,UNNEST(product_ids) AS product_id
    FROM
        orders AS t
    WHERE
        t.order_id NOT IN (
            SELECT t.order_id FROM order_id_cancel AS t)
),
--определяем 10 самых популярных товаров в таблице orders
ten_most_popular_products AS (
SELECT
    t.product_id
    ,COUNT(product_id) AS times_purchased
FROM
    unnest_table AS t
GROUP BY
    t.product_id
ORDER BY
    COUNT(product_id) DESC
LIMIT 10
)
--сортируем десять самых популярных товаров по id товара
SELECT
    t.product_id
    ,t.times_purchased
FROM
    ten_most_popular_products AS t
ORDER BY
    product_id ASC;

In [ ]:
###

In [ ]:
--23
--определим id (product_id) пяти самых дорогих товаров
WITH most_expensive_products AS (
SELECT
    t.product_id AS most_expensive_product_id
FROM
    products AS t
ORDER BY
    t.price DESC
LIMIT 5),
--с помощью UNNEST разворачиваем списки товаров в заказе,
--указанные в признаке product_ids таблицы orders
unnest_table AS (
SELECT
    t.order_id
    ,t.product_ids
    ,UNNEST(t.product_ids) AS product_id
FROM
    orders AS t
),
--отбираем order_id заказов, в которые входит
--хотя бы один из пяти самых дорогих товаров
selected_order_id AS (
SELECT
    DISTINCT(t.order_id) AS select_order_id
FROM
    unnest_table AS t
WHERE
    t.product_id IN (
        SELECT t.most_expensive_product_id
        FROM most_expensive_products AS t)
)
--выведем id  и содержимое заказов,
--включающих хотя бы один из пяти самых дорогих товаров
SELECT
    t.order_id
    ,t.product_ids
FROM
    orders AS t
WHERE
    t.order_id IN (
        SELECT t.select_order_id
        FROM selected_order_id AS t
    )
ORDER BY
    t.order_id;